In [1]:
using BenchmarkTools
using PyCall

Let's look at and benchmark the sum function:

$$\mathrm{sum}(x) = \sum_{i=1}^n x_i$$

In [2]:
x = rand(10^7);

In [3]:
sum(x)

4.999556292214221e6

In [4]:
d = Dict() # to store the measurement results

Dict{Any,Any}()

## Hand-written C

In [5]:
c_code = """
#include <stddef.h>
double c_sum(size_t n, double *X) {
    double s = 0.0;
    for (size_t i = 0; i < n; ++i) {
        s += X[i];
    }
    return s;
}
""";

In [6]:
# compile to a shared library by piping C_code to gcc:
# (only works if you have gcc installed)
const Clib = tempname()
using Libdl

In [7]:
open(`gcc -fPIC -O3 -msse3 -xc -shared -o $(Clib * "." * Libdl.dlext) -`, "w") do f
    print(f, c_code)
end

In [8]:
c_sum(X::Array{Float64}) = ccall(("c_sum", Clib), Float64, (Csize_t, Ptr{Float64}), length(X), X)

c_sum (generic function with 1 method)

In [9]:
c_sum(x) ≈ sum(x)

true

In [10]:
b = @benchmark c_sum($x)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     10.222 ms (0.00% GC)
  median time:      12.351 ms (0.00% GC)
  mean time:        13.038 ms (0.00% GC)
  maximum time:     42.979 ms (0.00% GC)
  --------------
  samples:          384
  evals/sample:     1

In [11]:
d["C"] = minimum(b.times) / 1e6

10.221516

## Hand-written C with -fast-math

In [12]:
const Clib_fastmath = tempname()   # make a temporary file

# The same as above but with a -ffast-math flag added
open(`gcc -fPIC -O3 -msse3 -xc -shared -ffast-math -o $(Clib_fastmath * "." * Libdl.dlext) -`, "w") do f
    print(f, c_code) 
end

# define a Julia function that calls the C function:
c_sum_fastmath(X::Array{Float64}) = ccall(("c_sum", Clib_fastmath), Float64, (Csize_t, Ptr{Float64}), length(X), X)

c_sum_fastmath (generic function with 1 method)

In [13]:
b = @benchmark c_sum_fastmath($x)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     4.308 ms (0.00% GC)
  median time:      4.841 ms (0.00% GC)
  mean time:        5.743 ms (0.00% GC)
  maximum time:     10.746 ms (0.00% GC)
  --------------
  samples:          871
  evals/sample:     1

In [14]:
d["C (fastmath)"] = minimum(b.times) / 1e6

4.308035

## Built-in Python `sum`

In [15]:
# get the Python built-in "sum" function:
pysum = pybuiltin("sum")

PyObject <built-in function sum>

In [16]:
# call a low-level PyCall function to get a Python list, because
# by default PyCall will convert to a NumPy array instead (we benchmark NumPy below):
xpy_list = PyCall.array2py(x);

In [17]:
b = @benchmark $pysum($xpy_list)

BenchmarkTools.Trial: 
  memory estimate:  32 bytes
  allocs estimate:  2
  --------------
  minimum time:     55.168 ms (0.00% GC)
  median time:      57.266 ms (0.00% GC)
  mean time:        59.170 ms (0.00% GC)
  maximum time:     74.765 ms (0.00% GC)
  --------------
  samples:          85
  evals/sample:     1

In [18]:
d["Python (built-in)"] = minimum(b.times) / 1e6

55.168154

## numpy `sum`

In [19]:
numpy_sum = pyimport("numpy").sum
b = @benchmark $numpy_sum($x)

BenchmarkTools.Trial: 
  memory estimate:  336 bytes
  allocs estimate:  6
  --------------
  minimum time:     3.583 ms (0.00% GC)
  median time:      3.753 ms (0.00% GC)
  mean time:        3.895 ms (0.00% GC)
  maximum time:     8.030 ms (0.00% GC)
  --------------
  samples:          1284
  evals/sample:     1

In [20]:
d["Python (numpy)"] = minimum(b.times) / 1e6

3.582819

## Hand-written Python

In [21]:
py"""
def mysum(a):
    s = 0.0
    for x in a:
        s = s + x
    return s
"""
mysum_py = py"mysum"

PyObject <function mysum at 0x7febbcbcbd40>

In [22]:
b = @benchmark $mysum_py($xpy_list)

BenchmarkTools.Trial: 
  memory estimate:  32 bytes
  allocs estimate:  2
  --------------
  minimum time:     212.160 ms (0.00% GC)
  median time:      217.512 ms (0.00% GC)
  mean time:        219.573 ms (0.00% GC)
  maximum time:     240.518 ms (0.00% GC)
  --------------
  samples:          23
  evals/sample:     1

In [23]:
d["Python (hand-written)"] = minimum(b.times) / 1e6

212.159834

## Built-in Julia `sum`

In [24]:
b = @benchmark sum($x)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     4.089 ms (0.00% GC)
  median time:      5.020 ms (0.00% GC)
  mean time:        5.592 ms (0.00% GC)
  maximum time:     11.051 ms (0.00% GC)
  --------------
  samples:          894
  evals/sample:     1

In [25]:
d["Julia (built-in)"] = minimum(b.times) / 1e6

4.089137

## Built-in Julia `sum` with  `Vector{Any}`

In [26]:
x_any = Vector{Any}(x)
b = @benchmark sum($x_any)

BenchmarkTools.Trial: 
  memory estimate:  152.59 MiB
  allocs estimate:  9999999
  --------------
  minimum time:     184.982 ms (8.30% GC)
  median time:      197.987 ms (8.05% GC)
  mean time:        201.038 ms (7.64% GC)
  maximum time:     251.713 ms (8.63% GC)
  --------------
  samples:          25
  evals/sample:     1

In [27]:
d["Julia (built-in, Any)"] = minimum(b.times) / 1e6

184.982358

## Hand-written Julia

In [28]:
function mysum(A)
    s = zero(eltype(A)) # the correct type of zero for A
    for a in A
        s += a
    end
    return s
end

mysum (generic function with 1 method)

In [29]:
b = @benchmark mysum($x)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     11.616 ms (0.00% GC)
  median time:      11.778 ms (0.00% GC)
  mean time:        11.927 ms (0.00% GC)
  maximum time:     14.069 ms (0.00% GC)
  --------------
  samples:          420
  evals/sample:     1

In [30]:
d["Julia (hand-written)"] = minimum(b.times) / 1e6

11.616404

## Hand-written Julia with `@simd`

In [31]:
function mysum_simd(A)
    s = zero(eltype(A)) # the correct type of zero for A
    @simd for a in A
        s += a
    end
    return s
end

mysum_simd (generic function with 1 method)

In [32]:
b = @benchmark mysum_simd($x)

BenchmarkTools.Trial: 
  memory estimate:  0 bytes
  allocs estimate:  0
  --------------
  minimum time:     4.185 ms (0.00% GC)
  median time:      5.002 ms (0.00% GC)
  mean time:        5.007 ms (0.00% GC)
  maximum time:     6.648 ms (0.00% GC)
  --------------
  samples:          999
  evals/sample:     1

In [33]:
d["Julia (hand-written, simd)"] = minimum(b.times) / 1e6

4.185407

## Summary

In [34]:
for (key, value) in sort(collect(d), by=x->x[2])
    println(rpad(key, 30, "."), lpad(round(value, digits=2), 10, "."))
end

Python (numpy)......................3.58
Julia (built-in)....................4.09
Julia (hand-written, simd)..........4.19
C (fastmath)........................4.31
C..................................10.22
Julia (hand-written)...............11.62
Python (built-in)..................55.17
Julia (built-in, Any).............184.98
Python (hand-written).............212.16


And of course, our hand-written Julia implementation is type-generic!

In [35]:
z = rand(Complex{Float64}, length(x));
@btime mysum_simd($z);

  8.682 ms (0 allocations: 0 bytes)


# How about other functions?

### Log

In [6]:
using BenchmarkTools

# uses the system C library
clog(x) = ccall(:log, Float64, (Float64,), x)
# uses LLVM's log
llvmlog(x) =  ccall(Symbol("llvm.log.f64"), llvmcall, Float64, (Float64,), x)

@btime log($(Ref(1.2))[])    
@btime clog($(Ref(1.2))[])    
@btime llvmlog($(Ref(1.2))[]);

  5.273 ns (0 allocations: 0 bytes)
  4.647 ns (0 allocations: 0 bytes)
  4.295 ns (0 allocations: 0 bytes)


### Exp

In [8]:
using BenchmarkTools

# uses the system C library
cexp(x) = ccall(:exp, Float64, (Float64,), x)
# uses LLVM's
llvmexp(x) =  ccall(Symbol("llvm.exp.f64"), llvmcall, Float64, (Float64,), x)

@btime exp($(Ref(1.2))[])    
@btime cexp($(Ref(1.2))[])    
@btime llvmexp($(Ref(1.2))[]);

  7.357 ns (0 allocations: 0 bytes)
  4.839 ns (0 allocations: 0 bytes)
  4.967 ns (0 allocations: 0 bytes)


(`6.037 ns` on Julia 1.6)

### Matrix multiplication

In [4]:
using BenchmarkTools

function A_mul_B!(𝐂, 𝐀, 𝐁)
   @inbounds for m ∈ axes(𝐀,1), n ∈ axes(𝐁,2)
       𝐂mn = zero(eltype(𝐂))
       for k ∈ axes(𝐀,2)
           𝐂mn += 𝐀[m,k] * 𝐁[k,n]
       end
       𝐂[m,n] = 𝐂mn
   end
end

A_mul_B! (generic function with 1 method)

In [5]:
N = 10
C = zeros(N,N);
A = rand(N,N);
B = rand(N,N);

In [6]:
@btime A_mul_B!($C,$A,$B);

  647.309 ns (0 allocations: 0 bytes)


[LoopVectorization.jl](https://github.com/chriselrod/LoopVectorization.jl)

In [9]:
using LoopVectorization

function A_mul_B_avx!(𝐂, 𝐀, 𝐁)
   @avx for m ∈ axes(𝐀,1), n ∈ axes(𝐁,2)
       𝐂mn = zero(eltype(𝐂))
       for k ∈ axes(𝐀,2)
           𝐂mn += 𝐀[m,k] * 𝐁[k,n]
       end
       𝐂[m,n] = 𝐂mn
   end
end

A_mul_B_avx! (generic function with 1 method)

In [10]:
@btime A_mul_B_avx!($C,$A,$B);

  80.862 ns (0 allocations: 0 bytes)


In [11]:
using LinearAlgebra

@btime mul!($C, $A, $B); # calls underlying BLAS

  256.278 ns (0 allocations: 0 bytes)


In [12]:
c_code = """
#include <stddef.h>
#include <math.h>

void gemm_mnk(double* restrict C, double* restrict A, double* restrict B, long M, long K, long N){
  for (long i = 0; i < M*N; i++){
    C[i] = 0.0;
  }
  for (long m = 0; m < M; m++){
    for (long n = 0; n < N; n++){
      for (long k = 0; k < K; k++){
	C[m + n*M] += A[m + k*M] * B[k + n*K];
      }
    }
  }
  return;
}
void gemm_mkn(double* restrict C, double* restrict A, double* restrict B, long M, long K, long N){
  for (long i = 0; i < M*N; i++){
    C[i] = 0.0;
  }
  for (long m = 0; m < M; m++){
    for (long k = 0; k < K; k++){
      for (long n = 0; n < N; n++){
	C[m + n*M] += A[m + k*M] * B[k + n*K];
      }
    }
  }
  return;
}
void gemm_nmk(double* restrict C, double* restrict A, double* restrict B, long M, long K, long N){
  for (long i = 0; i < M*N; i++){
    C[i] = 0.0;
  }
  for (long n = 0; n < N; n++){
    for (long m = 0; m < M; m++){
      for (long k = 0; k < K; k++){
	C[m + n*M] += A[m + k*M] * B[k + n*K];
      }
    }
  }
  return;
}
void gemm_nkm(double* restrict C, double* restrict A, double* restrict B, long M, long K, long N){
  for (long i = 0; i < M*N; i++){
    C[i] = 0.0;
  }
  for (long n = 0; n < N; n++){
    for (long k = 0; k < K; k++){
      for (long m = 0; m < M; m++){
	C[m + n*M] += A[m + k*M] * B[k + n*K];
      }
    }
  }
  return;
}
void gemm_kmn(double* restrict C, double* restrict A, double* restrict B, long M, long K, long N){
  for (long i = 0; i < M*N; i++){
    C[i] = 0.0;
  }
  for (long k = 0; k < K; k++){
    for (long m = 0; m < M; m++){
      for (long n = 0; n < N; n++){
	C[m + n*M] += A[m + k*M] * B[k + n*K];
      }
    }
  }
  return;
}
void gemm_knm(double* restrict C, double* restrict A, double* restrict B, long M, long K, long N){
  for (long i = 0; i < M*N; i++){
    C[i] = 0.0;
  }
  for (long k = 0; k < K; k++){
    for (long n = 0; n < N; n++){
      for (long m = 0; m < M; m++){
	C[m + n*M] += A[m + k*M] * B[k + n*K];
      }
    }
  }
  return;
}
""";

In [13]:
# compile to a shared library by piping C_code to gcc:
# (only works if you have gcc installed)
const Clib = tempname()
using Libdl

open(`gcc -fPIC -O3 -msse3 -xc -shared -o $(Clib * "." * Libdl.dlext) -`, "w") do f
    print(f, c_code)
end

c_gemm_mnk(C::Array{Float64},A::Array{Float64},B::Array{Float64}) = ccall(("gemm_mnk", Clib), Cvoid, (Ptr{Float64}, Ptr{Float64}, Ptr{Float64}, Clong, Clong, Clong), C, A, B, size(A,1), size(A,2), size(B, 2))

c_gemm_mkn(C::Array{Float64},A::Array{Float64},B::Array{Float64}) = ccall(("gemm_mkn", Clib), Cvoid, (Ptr{Float64}, Ptr{Float64}, Ptr{Float64}, Clong, Clong, Clong), C, A, B, size(A,1), size(A,2), size(B, 2))

c_gemm_nmk(C::Array{Float64},A::Array{Float64},B::Array{Float64}) = ccall(("gemm_nmk", Clib), Cvoid, (Ptr{Float64}, Ptr{Float64}, Ptr{Float64}, Clong, Clong, Clong), C, A, B, size(A,1), size(A,2), size(B, 2))

c_gemm_nkm(C::Array{Float64},A::Array{Float64},B::Array{Float64}) = ccall(("gemm_nkm", Clib), Cvoid, (Ptr{Float64}, Ptr{Float64}, Ptr{Float64}, Clong, Clong, Clong), C, A, B, size(A,1), size(A,2), size(B, 2))

c_gemm_kmn(C::Array{Float64},A::Array{Float64},B::Array{Float64}) = ccall(("gemm_kmn", Clib), Cvoid, (Ptr{Float64}, Ptr{Float64}, Ptr{Float64}, Clong, Clong, Clong), C, A, B, size(A,1), size(A,2), size(B, 2))

c_gemm_knm(C::Array{Float64},A::Array{Float64},B::Array{Float64}) = ccall(("gemm_knm", Clib), Cvoid, (Ptr{Float64}, Ptr{Float64}, Ptr{Float64}, Clong, Clong, Clong), C, A, B, size(A,1), size(A,2), size(B, 2))

c_gemm_knm (generic function with 1 method)

In [14]:
C2 = zeros(N,N);
c_gemm_mnk(C2, A, B)
@assert C ≈ C2
c_gemm_mkn(C2, A, B)
@assert C ≈ C2
c_gemm_nmk(C2, A, B)
@assert C ≈ C2
c_gemm_nkm(C2, A, B)
@assert C ≈ C2
c_gemm_kmn(C2, A, B)
@assert C ≈ C2
c_gemm_knm(C2, A, B)
@assert C ≈ C2

In [15]:
@btime c_gemm_mnk($C2, $A, $B)
@btime c_gemm_mkn($C2, $A, $B)
@btime c_gemm_nmk($C2, $A, $B)
@btime c_gemm_nkm($C2, $A, $B)
@btime c_gemm_kmn($C2, $A, $B)
@btime c_gemm_knm($C2, $A, $B)

  473.518 ns (0 allocations: 0 bytes)
  742.070 ns (0 allocations: 0 bytes)
  413.734 ns (0 allocations: 0 bytes)
  403.945 ns (0 allocations: 0 bytes)
  716.914 ns (0 allocations: 0 bytes)
  432.347 ns (0 allocations: 0 bytes)


**Note for larger `N`:** BLAS is multithreaded for larger `N`. In this case our `A_mul_B_avx!` is slower than `mul!`.

In [16]:
N = 100
C = zeros(N,N);
A = rand(N,N);
B = rand(N,N);

In [17]:
@btime A_mul_B_avx!($C, $A, $B);

  43.625 μs (0 allocations: 0 bytes)


In [18]:
@btime mul!($C, $A, $B);

  19.485 μs (0 allocations: 0 bytes)


However, [Octavian.jl](https://github.com/JuliaLinearAlgebra/Octavian.jl) is a (experimental) package that adds multithreading on top of LoopVectorization.jl. Again, we can blast OpenBLAS out of the water with pure Julia code. (Restart the kernel before executing the following.)

In [9]:
Threads.nthreads()

6

In [1]:
using Pkg
pkg"activate --temp"
pkg"add Octavian"
pkg"add BenchmarkTools"

In [2]:
using Octavian, LinearAlgebra, BenchmarkTools

In [5]:
N = 100; C = zeros(N,N); A = rand(N,N); B = rand(N,N);

In [6]:
@btime Octavian.matmul!($C, $A, $B);

  8.810 μs (0 allocations: 0 bytes)


In [7]:
@btime mul!($C, $A, $B);

  18.590 μs (0 allocations: 0 bytes)


**Resources/more info:**

https://github.com/mitmath/18S096/blob/master/lectures/lecture1/Performance-variation.ipynb

https://github.com/mitmath/18S096/blob/master/lectures/lecture1/Boxes-and-registers.ipynb


**More comprehensive benchmarks:** https://chriselrod.github.io/LoopVectorization.jl/stable/examples/matrix_multiplication/#Matrix-Multiplication